## Analyze the used car listings on Ebay Kleinanzeigen

In this project, we'll work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. It's an interesting dataset that I came across on Kaggle.

Dataset is available here: [Click here to download](https://www.kaggle.com/orgesleka/used-cars-database/data)

Out of the whole data, I have sampled 50,000 data points from the full dataset.

The main focus of this mini-project is to clean the data and **analyze the included used car listings.**

This would be done using `pandas `and `NumPy`.

Let's begin with importing both of the libraries.

In [1]:
import numpy as np
import pandas as pd

#reading the csv file
autos = pd.read_csv('autos.csv', encoding = 'Windows-1252')

In [2]:
autos.describe()

,yearOfRegistration,powerPS,monthOfRegistration,nrOfPictures,postalCode
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


In [3]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


There are total 50,000 rows and 20 columns. There are some null values and most of the columns are string datatype.

**1.) Cleaning Column Names**

- For this we would check the existing columns.
- As most of the columns has [camel case](https://en.wikipedia.org/wiki/Camel_case), we would convert that into [snake case](https://en.wikipedia.org/wiki/Snake_case).
- Along with this some changes in few wordings would be made to describe the columns more accurately.
- Then, we would update our `Dataframe`.

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
autos.rename(columns = {'yearOfRegistration' : 'registration_year',
                        'monthOfRegistration' : 'registration_month',
                        'notRepairedDamage' : 'unrepaired_damage',
                        'dateCreated' : 'ad_created',
                        'dateCrawled' : 'date_crawled', 
                        'offerType' : 'offer_type',
                        'abtest' : 'ab_test',
                        'vehicleType' : 'vehicle_type',
                        'powerPS' : 'power_ps',
                        'fuelType' : 'fuel_type',
                        'nrOfPictures' : 'num_photos',
                        'postalCode' : 'postal_code',
                        'lastSeen' : 'last_seen'}, inplace=True)
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen'],
      dtype='object')

Now after cleaning the columns let move to second task.

**2.) Initial Exploration and Cleaning.**

First need to explore the data to find obvious areas where one can clean the data.


In [6]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-10 15:36:24,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


- In here, one can see that `seller` and `offer type` contains all the same values.
- Along with this `num_photos` column values are mostly `NaN`, this needs further investigation.

In [7]:
print(autos['num_photos'].head())
autos['num_photos'].value_counts()

0    0
1    0
2    0
3    0
4    0
Name: num_photos, dtype: int64


0    50000
Name: num_photos, dtype: int64

As one can see, the `num_photos` column has `0` for every column. Hence, these three columns: `seller`, `offer_type` and `num_photos` will be dropped as all three contains one value.

In [8]:
autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)

`price` and `odometer`, contains numeric values with extra characters being stored as text. Next step is to clean and convert these.

In [9]:
autos['price'] = autos['price'].str.replace("$","")
autos['price'] = autos['price'].str.replace(",","")

autos['price'] = autos['price'].astype(int)

autos['price'].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [10]:
autos['odometer'] = autos['odometer'].str.replace("km","")
autos['odometer'] = autos['odometer'].str.replace(",","")

autos['odometer'] = autos['odometer'].astype(int)

autos.rename({'odometer' : 'odometer_km'},axis=1,inplace=True)
#autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)
autos['odometer_km'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

Now we have cleaned the data initially. After this, we would do the following task.

**3.) Exploring the Odometer and Price Columns**

Let's continue exploring the data, specifically looking for data that doesn't look right. We'll start by analyzing the `odometer_km` and `price` columns.

In [11]:
autos["odometer_km"].unique().shape

(13,)

In [12]:
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [13]:
autos["odometer_km"].value_counts().head().sort_index(ascending = False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
Name: odometer_km, dtype: int64

We can't see null or false data in the `odometer` column. There are 13 unique values Also, there are more vehicles with high mileages than low mileages.



In [14]:
autos["price"].shape

(50000,)

In [15]:
autos["price"].unique().shape

(2357,)

In [16]:
autos["price"].describe(include = 'all')

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [17]:
autos["price"].value_counts().sort_index(ascending = True)

0           1421
1            156
2              3
3              1
5              2
            ... 
10000000       1
11111111       2
12345678       3
27322222       1
99999999       1
Name: price, Length: 2357, dtype: int64

There are 2357 unique values. All values are mainly rounded that totally depends on the data entry operator how he enter the data. Whether he rounds the data or just fill the accurate data.
Also, there are 1421 values that have $0 price which is incorrect, so we will remove these values. Furthermore, highest value is one hundred million dollars, which sounds unreasonable or inaccurate data.

Let's inspect this highest data values.

In [18]:
autos["price"].value_counts().sort_index(ascending = False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [19]:
autos["price"].value_counts().sort_index(ascending = True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

We can see the price from highest to lowest one. Lowest can be `$1` because ebay is an auction site, but we will remove any value higher than `$350,000`.

Let's remove the outliers.

In [20]:
autos = autos.loc[autos["price"].between(1,351000),:]
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

In [21]:
autos['price'].shape

(48565,)

Here, we can see that the minimum and maximum values have been optimized.
Let's move to date columns and explore it.

**4.) Exploring the date columns**

In [22]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [23]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [24]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=False)

2016-04-07    0.001400
2016-04-06    0.003171
2016-04-05    0.013096
2016-04-04    0.036487
2016-04-03    0.038608
2016-04-02    0.035478
2016-04-01    0.033687
2016-03-31    0.031834
2016-03-30    0.033687
2016-03-29    0.034099
2016-03-28    0.034860
2016-03-27    0.031092
2016-03-26    0.032204
2016-03-25    0.031607
2016-03-24    0.029342
2016-03-23    0.032225
2016-03-22    0.032987
2016-03-21    0.037373
2016-03-20    0.037887
2016-03-19    0.034778
2016-03-18    0.012911
2016-03-17    0.031628
2016-03-16    0.029610
2016-03-15    0.034284
2016-03-14    0.036549
2016-03-13    0.015670
2016-03-12    0.036920
2016-03-11    0.032575
2016-03-10    0.032184
2016-03-09    0.033090
2016-03-08    0.033296
2016-03-07    0.036014
2016-03-06    0.014043
2016-03-05    0.025327
Name: date_crawled, dtype: float64

In here, Ebay site was crawled daily between March and April 2016. After normalizing data, it seems the distribution is normal.

In [25]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64

In [26]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=False)

2016-04-07    0.001256
2016-04-06    0.003253
2016-04-05    0.011819
2016-04-04    0.036858
2016-04-03    0.038855
                ...   
2015-12-05    0.000021
2015-11-10    0.000021
2015-09-09    0.000021
2015-08-10    0.000021
2015-06-11    0.000021
Name: ad_created, Length: 76, dtype: float64

There are many `ad created` dates. Mostly they are between 1-2 months of the listing date, but a few are quite old, with the oldest at around 9 months.

In [27]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

In [28]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=False)

2016-04-07    0.131947
2016-04-06    0.221806
2016-04-05    0.124761
2016-04-04    0.024483
2016-04-03    0.025203
2016-04-02    0.024915
2016-04-01    0.022794
2016-03-31    0.023783
2016-03-30    0.024771
2016-03-29    0.022341
2016-03-28    0.020859
2016-03-27    0.015649
2016-03-26    0.016802
2016-03-25    0.019211
2016-03-24    0.019767
2016-03-23    0.018532
2016-03-22    0.021373
2016-03-21    0.020632
2016-03-20    0.020653
2016-03-19    0.015834
2016-03-18    0.007351
2016-03-17    0.028086
2016-03-16    0.016452
2016-03-15    0.015876
2016-03-14    0.012602
2016-03-13    0.008895
2016-03-12    0.023783
2016-03-11    0.012375
2016-03-10    0.010666
2016-03-09    0.009595
2016-03-08    0.007413
2016-03-07    0.005395
2016-03-06    0.004324
2016-03-05    0.001071
Name: last_seen, dtype: float64

The date_crawled data provides us about any listing that it last saw. This helps us to determine on what day a listing was removed, with an assumption that the car was sold.

As you can seen the normalized `last seen` data which seems disproportionate at last three days. Its value is by far large from other normalized values.

It is very unlikely that there was a massive spike in sales and these values are more likley related to crawling period ending.

In [29]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

`registration_year` tells us about the day car is registered. This can be used to determine the age of the car. But as you can look at the maximum and minimum year it is false car can't be from rock age or future age.So, our next task would be dealing with these wrong registration date data.

**5.) Dealing with Incorrect Registration Year Data**


*   The minimum value is 1000, before cars were even born (just a pun).
*   The maximum value is 9999, many years into the future in the age of `'Jetsons'`.

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [30]:
#(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]

autos = autos[(autos["registration_year"]>=1900)& (autos["registration_year"]<=2016)]
autos['registration_year'].value_counts().sort_index(ascending = False)


2016    1220
2015     392
2014     663
2013     803
2012    1310
        ... 
1934       2
1931       1
1929       1
1927       1
1910       5
Name: registration_year, Length: 78, dtype: int64

One can see that we have removed the outliers and considered a possible year range from 1900 to 2016. Also, it appears that most of the vehicles were first registered in the past 20 years.

Now we move to the next step

**6.) Exploring Price by Brand**

In [31]:
brand_counts = autos['brand'].value_counts(normalize=True)
print(brand_counts)

#Displays top 5% of the overall listing
popular_brands = brand_counts[brand_counts > .05].index
print(popular_brands)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

In here the top list covers almost 50% of the overall listings. We will consider the further analysis with brands representing more than 5% of total listings as we can see lots of brands that don't have a significant percentage of listings. In here `Volkswagen` tops the chart followed by bmw and other famous brands.

In [32]:
mean_brand_prices= {}

for brand in popular_brands:
  common_brands = autos[autos['brand'] == brand]
  mean_prices = common_brands["price"].mean()
  mean_brand_prices[brand] = int(mean_prices) 



mean_brand_prices


{'audi': 9336,
 'bmw': 8332,
 'ford': 3749,
 'mercedes_benz': 8628,
 'opel': 2975,
 'volkswagen': 5402}

Of the top 5 brands, there is a distinct price gap:

* Audi, BMW and Mercedes Benz are more expensive
* Ford and Opel are more affordable.
* Volkswagen is in the middle of the both

This can be the reason it is more popular among general public.

**7.) Storing Aggregate Data in a DataFrame**

For the top 6 brands, let's use aggregation to understand the average mileage for those cars and if there's any visible link with mean price.

We can combine the data from both series objects into a single dataframe (with a shared index) and display the dataframe directly.

We can achieve this by using:


*   [pandas Series constructor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)
*   [pandas Dataframe constructor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)



In [33]:
#Using Series constructor
bmp_series = pd.Series(mean_brand_prices)
print(bmp_series)
#Using Dataframe constructor
df = pd.DataFrame(bmp_series, columns=['mean_price'])
print(df)

volkswagen       5402
bmw              8332
opel             2975
mercedes_benz    8628
audi             9336
ford             3749
dtype: int64
               mean_price
volkswagen           5402
bmw                  8332
opel                 2975
mercedes_benz        8628
audi                 9336
ford                 3749


In [34]:
mean_brand_mileage= {}

for brand in popular_brands:
  only_brand = autos[autos['brand'] == brand]
  mean_mileage = only_brand["odometer_km"].mean()
  mean_brand_mileage[brand] = int(mean_mileage) 

mileage_series = pd.Series(mean_brand_mileage).sort_values(ascending=False)
mean_series = pd.Series(mean_brand_prices).sort_values(ascending=False)
print(mean_brand_mileage)


{'volkswagen': 128707, 'bmw': 132572, 'opel': 129310, 'mercedes_benz': 130788, 'audi': 129157, 'ford': 124266}


In [35]:
brand_info = pd.DataFrame(mileage_series,columns=['mean_mileage'])
brand_info

,mean_mileage
bmw,132572
mercedes_benz,130788
opel,129310
audi,129157
volkswagen,128707
ford,124266


In [36]:
brand_info["mean_price"] = mean_series
brand_info

,mean_mileage,mean_price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


By aggregating, one can say that the expensive cars tend to provide higher mileage, while the less expensive ones have lower mileage. 

But one can also see there is not much variation in the `mileage` as one can see in `price` point.

**Challenge Questions:**
* Data cleaning next steps:
    - Identify categorical data that uses german words, translate them and map the values to their english counterparts
    - Convert the dates to be uniform numeric data, so "2016-03-21" becomes the integer 20160321.
    - See if there are particular keywords in the name column that you can extract as new columns

* Analysis next steps:
    - Find the most common brand/model combinations
    - Split the odometer_km into groups, and use aggregation to see if average prices follows any patterns based on the milage.
    - How much cheaper are cars with damage than their non-damaged counterparts?

Let's begin with determining german words and replacing them with english words.

In [37]:
autos.head()

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


In here, we can see german is present in following columns:
- `vehicle_type`
- `gearbox`
- `fuel_type`
- `unrepaired_damage`

Let's dig deeper into each column:

In [38]:
print('vehicle type: ',autos['vehicle_type'].unique())
print('gearbox: ',autos['gearbox'].unique())
print('fuel_type: ', autos['fuel_type'].unique())
print('unrepaired_damage: ', autos['unrepaired_damage'].unique())

vehicle type:  ['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']
gearbox:  ['manuell' 'automatik' nan]
fuel_type:  ['lpg' 'benzin' 'diesel' nan 'cng' 'hybrid' 'elektro' 'andere']
unrepaired_damage:  ['nein' nan 'ja']


We will change this names into english using dictionary.

In [39]:
# with reference to above values below are the translated words.
translated_words = {
    'bus':'bus',
    'limousine':'limousine',
    'kleinwagen':'supermini',
    'kombi':'station_wagon',
    'coupe':'coupe',
    'suv':'suv',
    'cabrio':'cabrio',
    'andere' :'other',
    'manuell':'manual',
    'automatik':'automatic',
    'lpg':'lpg',
    'benzin':'petrol',
    'diesel':'diesel',
    'cng':'cng',
    'hybrid':'hybrid',
    'elektro':'electro',
    'nein':'no',
    'ja':'yes'
}
for value in ['vehicle_type','gearbox','fuel_type','unrepaired_damage']:
    autos[value] = autos[value].map(translated_words) # using map function to replace the values.
    

Now we again check whether the values have changed or not.

In [40]:
print('vehicle type: ',autos['vehicle_type'].unique())
print('gearbox: ',autos['gearbox'].unique())
print('fuel_type: ', autos['fuel_type'].unique())
print('unrepaired_damage: ', autos['unrepaired_damage'].unique())

vehicle type:  ['bus' 'limousine' 'supermini' 'station_wagon' nan 'coupe' 'suv' 'cabrio'
 'other']
gearbox:  ['manual' 'automatic' nan]
fuel_type:  ['lpg' 'petrol' 'diesel' nan 'cng' 'hybrid' 'electro' 'other']
unrepaired_damage:  ['no' nan 'yes']


Our next task is to convert the dates to be uniform numeric data.

In [41]:
autos.head()

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manual,158,andere,150000,3,lpg,peugeot,no,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatic,286,7er,150000,6,petrol,bmw,no,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manual,102,golf,70000,7,petrol,volkswagen,no,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,supermini,2007,automatic,71,fortwo,70000,6,petrol,smart,no,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,station_wagon,2003,manual,0,focus,150000,7,petrol,ford,no,2016-04-01 00:00:00,39218,2016-04-01 14:38:50


In here one can see there are 3 columns with dates: `date_crawled`, `ad_created` and `last_seen`. As the date data is almost identical

For each column one can perform below steps:

- extract first nine parts
- remove "-" sign
- convert the remaining data to numeric format

In [42]:
date_cols = ['date_crawled','ad_created','last_seen']

for value in date_cols:
    autos[value] = (autos[value]
                  .str[:10]
                  .str.replace('-','')
                  .astype(int)
                  )

In [43]:
autos['date_crawled'].head()

0    20160326
1    20160404
2    20160326
3    20160312
4    20160401
Name: date_crawled, dtype: int64

One can see the dates have been converted into numeric format.

In [44]:
autos.head(20)

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,20160326,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manual,158,andere,150000,3,lpg,peugeot,no,20160326,79588,20160406
1,20160404,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatic,286,7er,150000,6,petrol,bmw,no,20160404,71034,20160406
2,20160326,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manual,102,golf,70000,7,petrol,volkswagen,no,20160326,35394,20160406
3,20160312,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,supermini,2007,automatic,71,fortwo,70000,6,petrol,smart,no,20160312,33729,20160315
4,20160401,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,station_wagon,2003,manual,0,focus,150000,7,petrol,ford,no,20160401,39218,20160401
5,20160321,Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...,7900,test,bus,2006,automatic,150,voyager,150000,4,diesel,chrysler,NaN,20160321,22962,20160406
6,20160320,VW_Golf_III_GT_Special_Electronic_Green_Metall...,300,test,limousine,1995,manual,90,golf,150000,8,petrol,volkswagen,NaN,20160320,31535,20160323
7,20160316,Golf_IV_1.9_TDI_90PS,1990,control,limousine,1998,manual,90,golf,150000,12,diesel,volkswagen,no,20160316,53474,20160407
8,20160322,Seat_Arosa,250,test,NaN,2000,manual,0,arosa,150000,10,NaN,seat,no,20160322,7426,20160326
9,20160316,Renault_Megane_Scenic_1.6e_RT_Klimaanlage,590,control,bus,1997,manual,90,megane,150000,7,petrol,renault,no,20160316,15749,20160406


In [45]:
autos.tail(20)

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
49979,20160320,Volkswagen_Polo_1.6_TDI_Style,11000,test,supermini,2011,manual,90,polo,70000,11,diesel,volkswagen,no,20160320,48455,20160407
49980,20160312,Ford_Escort_Turnier_16V,400,control,station_wagon,1995,manual,105,escort,125000,3,petrol,ford,NaN,20160312,56218,20160406
49981,20160315,Opel_Astra_Kombi_mit_Anhaengerkupplung,2000,control,station_wagon,1998,manual,115,astra,150000,12,petrol,opel,no,20160315,86859,20160405
49982,20160329,Skoda_Fabia_4_Tuerer_Bj:2004__85.000Tkm,1950,control,supermini,2004,manual,0,fabia,90000,7,petrol,skoda,NaN,20160329,45884,20160329
49983,20160306,Ford_focus_99,600,test,supermini,1999,manual,101,focus,150000,4,petrol,ford,NaN,20160306,52477,20160309
49985,20160402,Verkaufe_meinen_vw_vento!,1000,control,NaN,1995,automatic,0,NaN,150000,0,petrol,volkswagen,NaN,20160402,30900,20160406
49986,20160404,Chrysler_300C_3.0_CRD_DPF_Automatik_Voll_Ausst...,15900,control,limousine,2010,automatic,218,300c,125000,11,diesel,chrysler,no,20160404,73527,20160406
49987,20160322,Audi_A3_Limousine_2.0_TDI_DPF_Ambition__NAVI__...,21990,control,limousine,2013,manual,150,a3,50000,11,diesel,audi,no,20160322,94362,20160326
49988,20160328,BMW_330_Ci,9550,control,coupe,2001,manual,231,3er,150000,10,petrol,bmw,no,20160328,83646,20160407
49989,20160311,VW_Polo_zum_Ausschlachten_oder_Wiederaufbau,150,test,supermini,1997,manual,0,polo,150000,5,petrol,volkswagen,yes,20160311,21244,20160312


I cannot find any particular keywords in the name column that you can extract as new columns.

**Find the most common brand/model combinations**

In [46]:
autos['brand_model'] = autos['brand'] + '/' + autos['model']
autos['brand_model'].value_counts()[:5]

volkswagen/golf      3707
bmw/3er              2615
volkswagen/polo      1609
opel/corsa           1592
volkswagen/passat    1349
Name: brand_model, dtype: int64

From this one can tell that Volkswagen's most common model is the Golf, with 3684 units, BMW is 3er with 2615 units, and so on.

**Next task:**
- Split the `odometer_km` into groups, and use aggregation to see if average prices follows any patterns based on the milage.

In [47]:
# Split into groups

group_1 = autos.loc[autos["odometer_km"]<50000,'odometer_km']
group_2 = autos.loc[autos["odometer_km"].between(50000,100000),'odometer_km']
group_3 = autos.loc[autos["odometer_km"].between(100000,125000),'odometer_km']
group_4 = autos.loc[autos["odometer_km"].between(125000,150000),'odometer_km']

# Use aggregation to calculate average prices

mileage_avg_prices = {}
groups = [group_1,group_2,group_3,group_4]
group_number = 1

for group in groups:
    avg_price = autos.loc[group.index, "price"].mean()
    mileage_avg_prices['group_'+str(group_number)] = avg_price
    group_number+=1
    
mileage_avg_prices
    

{'group_1': 15212.965714285714,
 'group_2': 10093.070477775136,
 'group_3': 6785.046420824295,
 'group_4': 4107.938641176807}

This confirms that average prices depend on the mileage. On average, cars with under 50000 miles (newer ones) cost about 15000 dollars, whereas those with over 125000 miles cost 4000 dollars.

**Next task:**

How much cheaper are cars with damage than their non-damaged counterparts?

In [48]:
autos['unrepaired_damage'].unique()

array(['no', nan, 'yes'], dtype=object)

In [49]:
#Find what models are/are not damaged
damaged_models = autos.loc[autos['unrepaired_damage']=='yes',"model"]
non_damaged_models = autos.loc[autos['unrepaired_damage']=='no',"model"]

#Get the price for those models
damaged_prices = autos.loc[damaged_models.index,"price"]
non_damaged_prices = autos.loc[non_damaged_models.index,"price"]

#check damage_price
damaged_prices.describe()

count     4540.000000
mean      2241.146035
std       3563.276478
min          1.000000
25%        500.000000
50%       1000.000000
75%       2500.000000
max      44200.000000
Name: price, dtype: float64

In [50]:
non_damaged_prices.describe()

count     33834.000000
mean       7164.033103
std       10078.475478
min           1.000000
25%        1800.000000
50%        4150.000000
75%        9000.000000
max      350000.000000
Name: price, dtype: float64

With this one can conclude unrepaired damage can significantly lower car prices. On average, damaged models are priced at around 2000 dollars up to close to 45000, while non-damaged are worth 7000 dollars, up to 100000.